# Dependencies

In [169]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf
import matplotlib.pyplot as plt

# Creating/Cleaning Training Data

In [170]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
training_df = pd.read_csv('training_data.csv', index_col=0)
training_df['Buy'] = ""
buy_classification = ['buy', 'outperform', 'positive']

for index, row in training_df.iterrows():
    ticker = row['Ticker'].lower()
    analyst_suggestions = pd.read_csv('../Data_Collection/Scrapers/Final_Analyst_Rating.csv', index_col=0)
    for inner_index, inner_row in analyst_suggestions.iterrows():
        if analyst_suggestions.at[inner_index, 'Ticker'].lower() == ticker:
            if analyst_suggestions.at[inner_index, 'Rating'].lower() in buy_classification:
                training_df.at[index, 'Buy'] = 1
            else:
                training_df.at[index, 'Buy'] = 0

# Scaling the data
Scaler = MinMaxScaler()
cols = [x for x in training_df.columns if x not in ['Buy', 'Ticker']]
training_df[cols] = Scaler.fit_transform(training_df[cols])

##### TEMPORARY ######
cols_to_change = [x for x in training_df.columns if x not in ['Buy', 'Ticker']]
for col in cols_to_change:
    training_df.fillna(value=training_df[col].mean(), inplace=True)
######################

# training_df.drop(['payoutRatio', 'regularMarketVolume', 'sharesShort', 'mostRecentQuarter', 'forwardEps'
#                  ,'bookValue', 'sharesShort', 'sharesPercentSharesOut', 'mostRecentQuarter', 'nextFiscalYearEnd'
#                  , 'enterpriseValue', 'sharesShortPriorMonth', 'shortPercentOfFloat', 'earningsQuarterlyGrowth'],
#                  axis=1, inplace=True)

In [192]:
X_total = training_df[[x for x in training_df.columns if x not in ['Buy', 'Ticker']]]
y_total = training_df['Buy']
X_train, X_test, y_train, y_test = train_test_split(X_total, y_total, test_size=0.33, random_state=42)

# Training the model - CNN

In [228]:
model = keras.Sequential([
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

In [230]:
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, shuffle=True, validation_data=(X_test, y_test), verbose=0)
score = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', score[1])

Test accuracy: 0.5
